In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1


In [2]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [4]:
import os
import openai
import gradio as gr
import pandas as pd

# Load OpenAI API Key
openai.api_key =""

# Function to screen resumes based on job criteria
def screen_resumes(csv_file, job_role, must_have_skills):
    try:
        # Load resumes
        df = pd.read_csv(csv_file.name)

        # Ensure resume text column exists
        if "resume" not in df.columns:
            return "Error: The CSV must have a 'resume' column containing resume text."

        shortlisted = []

        # Loop through each resume and evaluate with GPT
        for index, row in df.iterrows():
            resume_text = row["resume"]

            prompt = (
                f"You are a smart HR assistant. Based on the job role '{job_role}' and these required skills: "
                f"{must_have_skills}, decide if this resume is a good match.\n\nResume:\n{resume_text}\n\n"
                f"Reply with 'MATCH' or 'NO MATCH' and a short reason why."
            )

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}]
            )

            result = response["choices"][0]["message"]["content"].strip()
            if "MATCH" in result:
                shortlisted.append({
                    "Index": index,
                    "Result": result,
                    "Resume Snippet": resume_text[:300] + "..."
                })

        if not shortlisted:
            return "No suitable candidates found."

        # Return results
        return "\n\n".join(
            [f"Candidate {c['Index']}:\n{c['Result']}\n{c['Resume Snippet']}" for c in shortlisted]
        )

    except Exception as e:
        return f"Error screening resumes: {str(e)}"

# Gradio interface
iface = gr.Interface(
    fn=screen_resumes,
    inputs=[
        gr.File(label="Upload CSV with 'resume' Column", file_types=[".csv"]),
        gr.Textbox(label="Job Role (e.g. Data Scientist, Sales Executive)"),
        gr.Textbox(label="Must-Have Skills (comma separated)")
    ],
    outputs="text",
    title="📄 Resume Screener (CSV)",
    description="Upload a CSV with resumes and I’ll return the top candidates based on the role and skills you specify."
)

# Launch the screener
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33f02c5b13af5bf3a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
